In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import  shuffle
from sklearn.model_selection import train_test_split
 
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
ls

sample_data/  sonar.csv


In [ ]:
#define the one hot encode function
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
#Read the sonar dataset
df = pd.read_csv('sonar.csv')
print(len(df.columns))
X = df[df.columns[0:60]].values
y=df[df.columns[60]]
#encode the dependent variable containing categorical values
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
Y = one_hot_encode(y)
 
#Transform the data in training and testing
X,Y = shuffle(X,Y,random_state=1)
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.20, random_state=42)

61


In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((166, 60), (42, 60), (166, 2), (42, 2))

In [ ]:
EPOCHS = 50
BATCH_SIZE = 11
LEARNING_RATE = 0.001

In [ ]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(60, 30) 
        self.layer_2 = nn.Linear(30, 64)
        self.layer_out = nn.Linear(64, 2) 
        
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(p=0.1)
        # self.batchnorm1 = nn.BatchNorm1d(64)
        # self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        # x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        # x = self.batchnorm2(x)
        # x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
###################### OUTPUT ######################
cuda:0

cpu


In [ ]:
model = binaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# ###################### OUTPUT ######################
# binaryClassification(
#   (layer_1): Linear(in_features=166, out_features=64, bias=True)
#   (layer_2): Linear(in_features=64, out_features=64, bias=True)
#   (layer_out): Linear(in_features=64, out_features=2, bias=True)
#   (relu): ReLU()
#   (dropout): Dropout(p=0.1, inplace=False)
#   (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#   (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
# )

binaryClassification(
  (layer_1): Linear(in_features=60, out_features=30, bias=True)
  (layer_2): Linear(in_features=30, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)


In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        # y_batch = y_batch.squeeze(1)
        loss = criterion(  y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch )
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} ')

Epoch 001: | Loss: 0.28321 
Epoch 002: | Loss: 0.27083 
Epoch 003: | Loss: 0.25594 
Epoch 004: | Loss: 0.29444 
Epoch 005: | Loss: 0.31536 
Epoch 006: | Loss: 0.29162 
Epoch 007: | Loss: 0.26807 
Epoch 008: | Loss: 0.25227 
Epoch 009: | Loss: 0.25466 
Epoch 010: | Loss: 0.23324 
Epoch 011: | Loss: 0.31968 
Epoch 012: | Loss: 0.27576 
Epoch 013: | Loss: 0.23203 
Epoch 014: | Loss: 0.26600 
Epoch 015: | Loss: 0.20884 
Epoch 016: | Loss: 0.21713 
Epoch 017: | Loss: 0.20125 
Epoch 018: | Loss: 0.20804 
Epoch 019: | Loss: 0.19542 
Epoch 020: | Loss: 0.20732 
Epoch 021: | Loss: 0.20285 
Epoch 022: | Loss: 0.18347 
Epoch 023: | Loss: 0.20231 
Epoch 024: | Loss: 0.18947 
Epoch 025: | Loss: 0.21313 
Epoch 026: | Loss: 0.18378 
Epoch 027: | Loss: 0.17222 
Epoch 028: | Loss: 0.15910 
Epoch 029: | Loss: 0.15824 
Epoch 030: | Loss: 0.15256 
Epoch 031: | Loss: 0.15860 
Epoch 032: | Loss: 0.14718 
Epoch 033: | Loss: 0.14450 
Epoch 034: | Loss: 0.14252 
Epoch 035: | Loss: 0.14050 
Epoch 036: | Loss: 0

In [ ]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
PATH = 'lstmmodelgpu.pth'
torch.save(model, PATH)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type binaryClassification. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
ls

lstmmodelgpu.pth  PYTORCHMODEL  sample_data/  sonar.csv


In [ ]:
model = torch.load(PATH)
model.eval()

binaryClassification(
  (layer_1): Linear(in_features=60, out_features=30, bias=True)
  (layer_2): Linear(in_features=30, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)

In [ ]:
model.weight

AttributeError: ignored

In [ ]:
torch.save(model.state_dict(),'best_detector.pth')


In [ ]:
model.eval()

binaryClassification(
  (layer_1): Linear(in_features=60, out_features=30, bias=True)
  (layer_2): Linear(in_features=30, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)

In [ ]:
model.load_state_dict(torch.load("best_detector.pth"))


<All keys matched successfully>

In [ ]:
model.parameters

<bound method Module.parameters of binaryClassification(
  (layer_1): Linear(in_features=60, out_features=30, bias=True)
  (layer_2): Linear(in_features=30, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)>

In [ ]:
test = model(X_test)

AttributeError: ignored